<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/advanced_topics/idc_dicomweb_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDC DICOMweb data access for pathology whole-slide images

This notebook accompanies the documentation article [IDC DICOMweb data access
](tbd) for the NCI Imaging Data Commons, which describes how one can access IDC pathology data using DICOMweb.

This notebook solely replicates the content of the code cells from the documentation article above for convenience. Please read the actual article for details!

Questions? Ask those at the IDC forum here: https://discourse.canceridc.dev.

-----
Created: July 2025

In [ ]:
%%capture
!pip install idc-index --upgrade
!pip install wsidicom
!pip install ez-wsi-dicomweb

## Unique identifiers: locating the relevant slides

In [ ]:
from idc_index import IDCClient

# Instantiate the client
idc_client = IDCClient()
idc_client.fetch_index('sm_index')


# Filter the slides
query = """
SELECT index.StudyInstanceUID, sm_index.SeriesInstanceUID
FROM sm_index
JOIN index ON sm_index.SeriesInstanceUID = index.SeriesInstanceUID
WHERE Modality = 'SM' AND primaryAnatomicStructure_CodeMeaning = 'Pancreas'
"""

pancreas_slides = idc_client.sql_query(query)

sample_study_uid = pancreas_slides['StudyInstanceUID'][0]
sample_series_uid = pancreas_slides['SeriesInstanceUID'][0]
sample_study_uid, sample_series_uid

## wsidicom

In [ ]:
from dicomweb_client.api import DICOMwebClient
from dicomweb_client.ext.gcp.session_utils import create_session_from_gcp_credentials

###If accessing Google-maintained DICOM store run the following cell

In [ ]:
# If accessing the Google-maintained DICOM store, you need to authenticate with your Google credentials first
from google.colab import auth
auth.authenticate_user()

# Create session and build path to DICOM store
session = create_session_from_gcp_credentials()

# Set-up a DICOMwebClient using the dicomweb_client library
google_dicom_store_url = 'https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v20/dicomWeb'
dw_client = DICOMwebClient(
    url=google_dicom_store_url,
    session = session
)

###If accessing IDC-maintained DICOM store run the following cell

In [ ]:
# Set-up a DICOMwebClient using the dicomweb_client library
idc_dicom_store_url = 'https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb'
dw_client = DICOMwebClient(url=idc_dicom_store_url)

### Access

In [ ]:
import wsidicom
import matplotlib.pyplot as plt

In [ ]:
# Wrap into wsidicom client
wsidicom_client = wsidicom.WsiDicomWebClient(dw_client)

# Open the slide via DICOMweb interface of wsidicom
slide = wsidicom.WsiDicom.open_web(wsidicom_client,
    study_uid = sample_study_uid,
    series_uids = sample_series_uid
)
# Investigate existing levels and their dimensions
print(slide)

In [ ]:
# Access and visualize 500x500px subregion at level 4, starting from pixel (1000,1000)
region = slide.read_region(location=(1000, 1000), level=4, size=(500, 500))
plt.imshow(region)
plt.show()

## ez-wsi-dicomweb

In [ ]:
from ez_wsi_dicomweb import dicomweb_credential_factory
from ez_wsi_dicomweb import dicom_slide
from ez_wsi_dicomweb import local_dicom_slide_cache_types
from ez_wsi_dicomweb import dicom_web_interface
from ez_wsi_dicomweb import patch_generator
from ez_wsi_dicomweb import pixel_spacing
from ez_wsi_dicomweb.ml_toolkit import dicom_path

In [ ]:
from google.colab import auth
auth.authenticate_user()

google_dicom_store_url = 'https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v21/dicomWeb'

series_path_str = (
      f'{google_dicom_store_url}'
      f'/studies/{sample_study_uid}'
      f'/series/{sample_series_uid}'
)
series_path = dicom_path.FromString(series_path_str)
dcf = dicomweb_credential_factory.CredentialFactory()
dwi = dicom_web_interface.DicomWebInterface(dcf)

ds = dicom_slide.DicomSlide(
    dwi=dwi,
    path=series_path,
    enable_client_slide_frame_decompression = True
    )

# Enable slide frame cache. More information: https://github.com/GoogleCloudPlatform/EZ-WSI-DICOMweb/blob/main/ez_wsi_demo.ipynb
ds.init_slide_frame_cache(
    optimization_hint=local_dicom_slide_cache_types.CacheConfigOptimizationHint.MINIMIZE_LATENCY
    )

In [ ]:
# Investigate existing levels and their dimensions
for level in ds.levels:
    print(f'Level {level.level_index} has pixel dimensions (row, col): {level.height, level.width}')

In [ ]:
# Access and visualize 500x500px subregion at level 3, starting from pixel (1000,1000)

level = ds.get_level_by_index(3)
region = ds.get_patch(level=level, x=1000, y=1000, width=500, height=500).image_bytes()
plt.imshow(region)
plt.show()